In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
import json
from datetime import datetime

In [2]:

class SequenceAlertPredictor:
    def __init__(self, model_type='xgboost', sequence_length=24):  # 24 hours of data
        """
        Initializes the SequenceAlertPredictor with the specified model type and sequence length.

        Args:
            model_type (str): Type of model to use ('xgboost' or 'randomforest')
            sequence_length (int): Number of time points to use for prediction
        """
        self.model_type = model_type
        self.sequence_length = sequence_length
        self.models = {}
        self.scalers = {}
        self.alert_types = ['LOW', 'MEDIUM', 'HIGH']
        self.base_features = ['ChlPrs', 'rolling_mean', 'rolling_std']
        self.features = []
        for feature in self.base_features:
            self.features.extend([f'{feature}_t{i}' for i in range(sequence_length)])
        self.features.extend([f'time_since_{at}' for at in self.alert_types])

    def load_and_preprocess_data(self, folder):
        """
        Loads and preprocesses data from CSV files in the specified folder.
        """
        dfs = []
        for i in range(9, 16):
            file_name = f"HTOL-{i:02d}_alerts.csv"
            df = pd.read_csv(os.path.join(folder, file_name))
            df['machine_id'] = f'HTOL-{i:02d}'
            dfs.append(df)

        combined_df = pd.concat(dfs, ignore_index=True)
        combined_df['Time'] = pd.to_datetime(combined_df['Time'])
        combined_df = combined_df.sort_values(['machine_id', 'Time'])

        return combined_df

    def create_sequences(self, df):
        """
        Creates sequences of data points for each machine.
        """
        sequences = []
        labels = []

        for machine_id in df['machine_id'].unique():
            machine_data = df[df['machine_id'] == machine_id].copy()

            for i in range(len(machine_data) - self.sequence_length):
                sequence = machine_data.iloc[i:i + self.sequence_length]
                target_row = machine_data.iloc[i + self.sequence_length]

                sequence_features = {}

                # Add sequential features
                for feature in self.base_features:
                    for j in range(self.sequence_length):
                        sequence_features[f'{feature}_t{j}'] = sequence.iloc[j][feature]

                # Add time since last alert features
                for alert_type in self.alert_types:
                    sequence_features[f'time_since_{alert_type}'] = target_row[f'time_since_{alert_type}']

                sequences.append(sequence_features)
                labels.append(1 if target_row['ALERT'] == self.current_alert_type else 0)

        return pd.DataFrame(sequences), np.array(labels)

    def engineer_features(self, df):
        """
        Engineers features from the preprocessed data.
        """
        df['rolling_mean'] = df.groupby('machine_id')['ChlPrs'].rolling(
            window=24, min_periods=1).mean().reset_index(0, drop=True)
        df['rolling_std'] = df.groupby('machine_id')['ChlPrs'].rolling(
            window=24, min_periods=1).std().reset_index(0, drop=True)

        for alert_type in self.alert_types:
            df[f'time_since_{alert_type}'] = df.groupby('machine_id').apply(
                lambda x: x['Time'] - x[x['ALERT'] == alert_type]['Time'].shift(1)
            ).reset_index(level=0, drop=True)
            df[f'time_since_{alert_type}'] = df[f'time_since_{alert_type}'].dt.total_seconds() / 3600

        return df

    def train_and_evaluate_classifier(self, X, y, test_size=0.2):
        """
        Trains and evaluates the classification model.
        """
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        if self.model_type == 'xgboost':
            model = XGBClassifier(
                n_estimators=100,
                learning_rate=0.1,
                max_depth=6,
                scale_pos_weight=len(y_train[y_train == 0]) / len(y_train[y_train == 1]),
                random_state=42
            )
        else:  # randomforest
            model = RandomForestClassifier(
                n_estimators=100,
                class_weight='balanced',
                random_state=42
            )

        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        print(classification_report(y_test, y_pred))

        return model, scaler

    def train(self, folder):
        """
        Trains models for each alert type.
        """
        df = self.load_and_preprocess_data(folder)
        df = self.engineer_features(df)

        for alert_type in self.alert_types:
            print(f"\nTraining model for {alert_type} alerts:")
            self.current_alert_type = alert_type
            X, y = self.create_sequences(df)
            model, scaler = self.train_and_evaluate_classifier(X, y)
            self.models[alert_type] = model
            self.scalers[alert_type] = scaler

In [3]:

class ProductionSequencePredictor:
    def __init__(self, model_types=['xgboost', 'randomforest'], sequence_length=24):
        """
        Initializes the ProductionSequencePredictor with multiple model types.
        """
        self.model_types = model_types
        self.sequence_length = sequence_length
        self.models = {model_type: {} for model_type in model_types}
        self.scalers = {model_type: {} for model_type in model_types}
        self.alert_types = ['LOW', 'MEDIUM', 'HIGH']
        self.base_features = ['ChlPrs', 'rolling_mean', 'rolling_std']
        self.features = []
        for feature in self.base_features:
            self.features.extend([f'{feature}_t{i}' for i in range(sequence_length)])
        self.features.extend([f'time_since_{at}' for at in self.alert_types])

    def prepare_sequence(self, df):
        """
        Prepares a sequence of data points for prediction.
        """
        if len(df) < self.sequence_length:
            raise ValueError(f"Input data must contain at least {self.sequence_length} points")

        sequence = df.iloc[-self.sequence_length:]
        sequence_features = {}

        for feature in self.base_features:
            for j in range(self.sequence_length):
                sequence_features[f'{feature}_t{j}'] = sequence.iloc[j][feature]

        for alert_type in self.alert_types:
            sequence_features[f'time_since_{alert_type}'] = df.iloc[-1][f'time_since_{alert_type}']

        return pd.DataFrame([sequence_features])

    def save_models(self, output_dir):
        """
        Saves trained models and metadata to disk.
        """
        os.makedirs(output_dir, exist_ok=True)

        metadata = {
            'model_types': self.model_types,
            'alert_types': self.alert_types,
            'sequence_length': self.sequence_length,
            'features': self.features,
            'timestamp': datetime.now().isoformat()
        }

        with open(os.path.join(output_dir, 'metadata.json'), 'w') as f:
            json.dump(metadata, f, indent=4)

        for model_type in self.model_types:
            model_dir = os.path.join(output_dir, model_type)
            os.makedirs(model_dir, exist_ok=True)

            for alert_type in self.alert_types:
                with open(os.path.join(model_dir, f'{alert_type}_model.pkl'), 'wb') as f:
                    pickle.dump(self.models[model_type][alert_type], f)
                with open(os.path.join(model_dir, f'{alert_type}_scaler.pkl'), 'wb') as f:
                    pickle.dump(self.scalers[model_type][alert_type], f)

    @classmethod
    def load_models(cls, model_dir):
        """
        Loads trained models from disk.
        """
        with open(os.path.join(model_dir, 'metadata.json'), 'r') as f:
            metadata = json.load(f)

        predictor = cls(
            model_types=metadata['model_types'],
            sequence_length=metadata['sequence_length']
        )

        for model_type in predictor.model_types:
            model_type_dir = os.path.join(model_dir, model_type)

            for alert_type in predictor.alert_types:
                with open(os.path.join(model_type_dir, f'{alert_type}_model.pkl'), 'rb') as f:
                    predictor.models[model_type][alert_type] = pickle.load(f)
                with open(os.path.join(model_type_dir, f'{alert_type}_scaler.pkl'), 'rb') as f:
                    predictor.scalers[model_type][alert_type] = pickle.load(f)

        return predictor

    def predict(self, df, threshold=0.7):
        """
        Makes predictions using the ensemble of models.
        """
        sequence = self.prepare_sequence(df)
        results = {}

        for alert_type in self.alert_types:
            model_predictions = []
            model_probabilities = []

            for model_type in self.model_types:
                X_scaled = self.scalers[model_type][alert_type].transform(sequence)
                probabilities = self.models[model_type][alert_type].predict_proba(X_scaled)[:, 1]
                predictions = (probabilities >= threshold).astype(int)

                model_predictions.append(predictions)
                model_probabilities.append(probabilities)

            # Unanimous ensemble prediction
            final_predictions = np.all(model_predictions, axis=0)
            avg_probabilities = np.mean(model_probabilities, axis=0)

            results[alert_type] = {
                'prediction': final_predictions[0],
                'probability': avg_probabilities[0],
                'model_probabilities': {
                    model_type: probs[0]
                    for model_type, probs in zip(self.model_types, model_probabilities)
                }
            }

        return results

In [4]:

def train_production_models(data_folder, output_dir, sequence_length=24):
    """
    Trains and saves production models.
    """
    xgb_predictor = SequenceAlertPredictor(model_type='xgboost', sequence_length=sequence_length)
    rf_predictor = SequenceAlertPredictor(model_type='randomforest', sequence_length=sequence_length)

    xgb_predictor.train(data_folder)
    rf_predictor.train(data_folder)

    prod_predictor = ProductionSequencePredictor(['xgboost', 'randomforest'], sequence_length)

    for alert_type in prod_predictor.alert_types:
        prod_predictor.models['xgboost'][alert_type] = xgb_predictor.models[alert_type]
        prod_predictor.scalers['xgboost'][alert_type] = xgb_predictor.scalers[alert_type]
        prod_predictor.models['randomforest'][alert_type] = rf_predictor.models[alert_type]
        prod_predictor.scalers['randomforest'][alert_type] = rf_predictor.scalers[alert_type]

    prod_predictor.save_models(output_dir)
    return prod_predictor

def visualize_predictions(predictor, df, alert_type, window_size=168):  # 1 week
    """
    Visualizes predictions vs actual alerts.
    """
    predictions = []
    actual_alerts = []
    timestamps = []

    for i in range(predictor.sequence_length, len(df)):
        sequence_df = df.iloc[max(0, i-window_size):i]
        if len(sequence_df) >= predictor.sequence_length:
            pred = predictor.predict(sequence_df)
            predictions.append(pred[alert_type]['probability'])
            actual_alerts.append(1 if df.iloc[i]['ALERT'] == alert_type else 0)
            timestamps.append(df.iloc[i]['Time'])

    plt.figure(figsize=(15, 6))
    plt.plot(timestamps, predictions, label='Prediction Probability', color='blue', alpha=0.6)
    plt.scatter([t for t, a in zip(timestamps, actual_alerts) if a == 1],
                [1 for a in actual_alerts if a == 1],
                color='red', label='Actual Alerts', marker='x', s=100)
    plt.axhline(y=0.7, color='r', linestyle='--', alpha=0.3, label='Threshold (0.7)')
    plt.xlabel('Time')
    plt.ylabel('Alert Probability')
    plt.title(f'Predicted vs Actual {alert_type} Alerts')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:

def train_production_models(data_folder, output_dir, sequence_length=24):
    """
    Trains and saves production models.
    """
    xgb_predictor = SequenceAlertPredictor(model_type='xgboost', sequence_length=sequence_length)
    rf_predictor = SequenceAlertPredictor(model_type='randomforest', sequence_length=sequence_length)

    xgb_predictor.train(data_folder)
    rf_predictor.train(data_folder)

    prod_predictor = ProductionSequencePredictor(['xgboost', 'randomforest'], sequence_length)

    for alert_type in prod_predictor.alert_types:
        prod_predictor.models['xgboost'][alert_type] = xgb_predictor.models[alert_type]
        prod_predictor.scalers['xgboost'][alert_type] = xgb_predictor.scalers[alert_type]
        prod_predictor.models['randomforest'][alert_type] = rf_predictor.models[alert_type]
        prod_predictor.scalers['randomforest'][alert_type] = rf_predictor.scalers[alert_type]

    prod_predictor.save_models(output_dir)
    return prod_predictor

def visualize_predictions(predictor, df, alert_type, window_size=168):  # 1 week
    """
    Visualizes predictions vs actual alerts.
    """
    predictions = []
    actual_alerts = []
    timestamps = []

    for i in range(predictor.sequence_length, len(df)):
        sequence_df = df.iloc[max(0, i-window_size):i]
        if len(sequence_df) >= predictor.sequence_length:
            pred = predictor.predict(sequence_df)
            predictions.append(pred[alert_type]['probability'])
            actual_alerts.append(1 if df.iloc[i]['ALERT'] == alert_type else 0)
            timestamps.append(df.iloc[i]['Time'])

    plt.figure(figsize=(15, 6))
    plt.plot(timestamps, predictions, label='Prediction Probability', color='blue', alpha=0.6)
    plt.scatter([t for t, a in zip(timestamps, actual_alerts) if a == 1],
                [1 for a in actual_alerts if a == 1],
                color='red', label='Actual Alerts', marker='x', s=100)
    plt.axhline(y=0.7, color='r', linestyle='--', alpha=0.3, label='Threshold (0.7)')
    plt.xlabel('Time')
    plt.ylabel('Alert Probability')
    plt.title(f'Predicted vs Actual {alert_type} Alerts')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
# Loading saved models and making predictions
predictor = ProductionSequencePredictor.load_models("production_sequence_models")

# Load test data
test_df = pd.read_csv("path_to_test_data.csv")
test_df['Time'] = pd.to_datetime(test_df['Time'])
test_df = test_df.sort_values('Time')

# Visualize predictions for each alert type
for alert_type in predictor.alert_types:
    visualize_predictions(predictor, test_df, alert_type)

In [ ]:
output_dir = "production_sequence_models"
data_folder = "../../../outlier_tolerance=5_grouping_time_window=200_anomaly_threshold=6_start_date=2022-01-01_end_date=2026-01-01"
sequence_length = 24  # 24 hours of data

predictor = train_production_models(
    data_folder=data_folder,
    output_dir=output_dir,
    sequence_length=sequence_length
)

/var/folders/d7/0np89js16x9b596pzk8m108c0000gn/T/ipykernel_43256/3248836662.py:78: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[f'time_since_{alert_type}'] = df.groupby('machine_id').apply(
/var/folders/d7/0np89js16x9b596pzk8m108c0000gn/T/ipykernel_43256/3248836662.py:78: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[f'time_since_{alert_type}'] = df.groupby('machine_id').apply(
/var/folders/d7/0np89j


Training model for LOW alerts:


In [ ]:
# Loading saved models and making predictions
predictor = ProductionSequencePredictor.load_models("production_sequence_models")

# Load test data
test_df = pd.read_csv("path_to_test_data.csv")
test_df['Time'] = pd.to_datetime(test_df['Time'])
test_df = test_df.sort_values('Time')

# Visualize predictions for each alert type
for alert_type in predictor.alert_types:
    visualize_predictions(predictor, test_df, alert_type)